# EDA

- The os module has a perfect method to list files in a directory.
- Pandas json normalize could work here but is not necessary to convert the JSON data to a dataframe.
- You may need a nested for-loop to access each sale!
- We've put a lot of time into creating the structure of this repository, and it's a good example for future projects.  In the file functions_variables.py, there is an example function that you can import and use.  If you have any variables, functions or classes that you want to make, they can be put in the functions_variables.py file and imported into a notebook.  Note that only .py files can be imported into a notebook. If you want to import everything from a .py file, you can use the following:
```python
from functions_variables import *
```
If you just import functions_variables, then each object from the file will need to be prepended with "functions_variables"\
Using this .py file will keep your notebooks very organized and make it easier to reuse code between notebooks.

In [16]:
# (this is not an exhaustive list of libraries)
import pandas as pd
import numpy as np
import os
import json
from pprint import pprint
from functions_variables import threshold_column, ohe_column, json_to_df


## Data Importing

In [17]:
# load one file first to see what type of data you're dealing with and what attributes it has
tmp_df = pd.read_json('../data/housing/AK_Juneau_0.json')
# tmp_df['data']['results']

In [18]:
# Create empty DataFrame for housing
df = pd.DataFrame()
# The path to the data files
dir_path = '../data/housing/'

In [19]:
# loop over all files and put them into a dataframe
# n = 0
for file_name in sorted(os.listdir(dir_path)):
    # n += 1
    if file_name.endswith('.json'):
        # print(f'File: {file_name}')
        file_path = os.path.join(dir_path, file_name)
        tmp = json_to_df(file_path)
        tmp = tmp.dropna(axis=1, how='all')
        df = pd.concat([df, tmp], ignore_index=True)
    # if n > 1: break
# df

## Data Cleaning and Wrangling

At this point, ensure that you have all sales in a dataframe.
- Take a quick look at your data (i.e. `.info()`, `.describe()`) - what do you see?
- Is each cell one value, or do some cells have lists?
- What are the data types of each column?
- Some sales may not actually include the sale price (target).  These rows should be dropped.
- There are a lot of NA/None values.  Should these be dropped or replaced with something?
    - You can drop rows or use various methods to fills NA's - use your best judgement for each column 
    - i.e. for some columns (like Garage), NA probably just means no Garage, so 0
- Drop columns that aren't needed
    - Don't keep the list price because it will be too close to the sale price. Assume we want to predict the price of houses not yet listed

In [20]:
df.shape

(8191, 50)

In [21]:
# df.info()

In [22]:
columns = df.columns.sort_values().to_list()
# columns

In [23]:
# Rearrange DataFrame columns
df = df[columns]

In [24]:
# load and concatenate data here


In [25]:
# drop or replace values as necessary
df = df.drop(
    columns=[
        'branding',
        'community.advertisers',
        'community.description.name',
        # 'description.baths',
        # 'description.baths_3qtr',
        # 'description.baths_full',
        # 'description.baths_half',
        # 'description.beds',
        # 'description.garage',
        # 'description.lot_sqft',
        # 'description.sold_date',
        # 'description.sold_price',
        # 'description.sqft',
        # 'description.stories',
        'description.sub_type',
        'description.type',
        # 'description.year_built',
        # 'flags.is_foreclosure',
        # 'flags.is_new_listing',
        # 'flags.is_price_reduced',
        'last_update_date',
        'lead_attributes.show_contact_an_agent',
        # 'list_date',
        # 'list_price',
        'listing_id',
        # 'location.address.city',
        # 'location.address.coordinate.lat',
        # 'location.address.coordinate.lon',
        'location.address.line',
        # 'location.address.postal_code',
        # 'location.address.state',
        # 'location.address.state_code',
        # 'location.county.fips_code',
        # 'location.county.name',
        'location.street_view_url',
        # 'matterport',
        'other_listings.rdc',
        'permalink',
        'photos',
        # 'price_reduced_amount',
        'primary_photo.href',
        # 'products.brand_name',
        # 'property_id',
        'source.agents',
        'source.plan_id',
        'source.spec_id',
        # 'source.type',
        # 'status',
        # 'tags',
        'virtual_tours'
    ],
    errors='ignore'
)

In [26]:
# Rename columns - to remove name prefixes
columns = {col: col.split('.')[-1] for col in df.columns}
# columns
df.rename(columns=columns, inplace=True)
df = df[sorted(df.columns)]
# df.head()

### Dealing with Tags

Consider the fact that with tags, there are a lot of categorical variables.
- How many columns would we have if we OHE tags, city and state?
- Perhaps we can get rid of tags that have a low frequency.

In [44]:
# OHE categorical variables / tags here
# tags will have to be done manually
df['tags'] = threshold_column(df['tags'], 'tags', 1000)
# df['city'] = threshold_column(df['city'], 'city', 100)
# df['state'] = threshold_column(df['state'], 'state', 10)

In [51]:
# One-Hot Encode the columns, considering low frequency removal for 'tags'

# Get dummy variables
dummies_tags = ohe_column(df['tags'], 'tags')
# dummies_city = ohe_column(df['city'], 'city')
# dummies_state = ohe_column(df['state'], 'state')
ohe_df = pd.concat([df, dummies_tags], axis=1)
# ohe_df = pd.concat([ohe_df, dummies_city], axis=1)
# ohe_df = pd.concat([ohe_df, dummies_state], axis=1)
ohe_df.drop(['tags'], axis=1, inplace=True)
# ohe_df.drop(['city'], axis=1, inplace=True)
# ohe_df.drop(['state'], axis=1, inplace=True)
ohe_df.shape

(59071, 55)

In [52]:
ohe_df.head()

,baths,baths_3qtr,baths_full,baths_half,beds,brand_name,city,fips_code,garage,is_foreclosure,...,garage_1_or_more,garage_2_or_more,hardwood_floors,laundry_room,recreation_facilities,shopping,single_story,two_or_more_stories,view,washer_dryer
0,2.0,NaN,2.0,NaN,3.0,basic_opt_in,Juneau,NaN,1.0,NaN,...,False,False,False,False,False,False,False,False,False,False
1,NaN,NaN,NaN,NaN,NaN,NaN,Juneau,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
2,NaN,NaN,NaN,NaN,NaN,NaN,Juneau,NaN,NaN,NaN,...,False,False,True,False,False,False,False,False,False,False
3,NaN,NaN,NaN,NaN,NaN,NaN,Juneau,NaN,NaN,NaN,...,False,False,False,False,True,False,False,False,False,False
4,NaN,NaN,NaN,NaN,NaN,NaN,Juneau,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False


### Dealing with Cities

- Sales will vary drastically between cities and states.  Is there a way to keep information about which city it is without OHE?
- Could we label encode or ordinal encode?  Yes, but this may have undesirable effects, giving nominal data ordinal values.
- What we can do is use our training data to encode the mean sale price by city as a feature (a.k.a. Target Encoding)
    - We can do this as long as we ONLY use the training data - we're using the available data to give us a 'starting guess' of the price for each city, without needing to encode city explicitly
- If you replace cities or states with numerical values (like the mean price), make sure that the data is split so that we don't leak data into the training selection. This is a great time to train test split. Compute on the training data, and join these values to the test data
- Note that you *may* have cities in the test set that are not in the training set. You don't want these to be NA, so maybe you can fill them with the overall mean

In [29]:
# perform train test split here
# do something with state and city

## Extra Data - STRETCH

> This doesn't need to be part of your Minimum Viable Product (MVP). We recommend you write a functional, basic pipeline first, then circle back and join new data if you have time

> If you do this, try to write your downstream steps in a way it will still work on a dataframe with different features!

- You're not limited to just using the data provided to you. Think/ do some research about other features that might be useful to predict housing prices. 
- Can you import and join this data? Make sure you do any necessary preprocessing and make sure it is joined correctly.
- Example suggestion: could mortgage interest rates in the year of the listing affect the price? 

In [30]:
# import, join and preprocess new data here

## EDA/ Visualization

Remember all of the EDA that you've been learning about?  Now is a perfect time for it!
- Look at distributions of numerical variables to see the shape of the data and detect outliers.    
    - Consider transforming very skewed variables
- Scatterplots of a numerical variable and the target go a long way to show correlations.
- A heatmap will help detect highly correlated features, and we don't want these.
    - You may have too many features to do this, in which case you can simply compute the most correlated feature-pairs and list them
- Is there any overlap in any of the features? (redundant information, like number of this or that room...)

In [31]:
# perform EDA here

## Scaling and Finishing Up

Now is a great time to scale the data and save it once it's preprocessed.
- You can save it in your data folder, but you may want to make a new `processed/` subfolder to keep it organized